In [ ]:
# ==============================================================================
# Python Scripts for K-Means Clustering Pipeline
#
# This file contains the complete data processing and analysis pipeline for the
# K-Means clustering analysis section of the thesis. It is organized into
# several sequential scripts:
#
# Script 1-4: Feature Engineering from raw data files.
# Script 5: Merging all features into a final matrix.
# Script 6: Performing K-Means clustering and robustness checks.
# Script 7-8: Validating the resulting clusters against course outcomes.
# Script 9: Standalone script for K-Means robustness check (50 runs).
#
# Author: Ziyun Ke
# Date: August 2025
# ==============================================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA

# ==============================================================================
# SCRIPT 1: FEATURE ENGINEERING - ASSESSMENT ENGAGEMENT
# ==============================================================================
print("--- Running Script 1: Feature Engineering - Assessment Engagement ---")

# --- 0. Configuration ---
# NOTE: This script needs to be run for each course by changing the input file.
input_file_assess = 'assessments_course-v1_KULeuvenX+EUROHISx+1T2023.csv'
output_file_assess = 'feature_assessment_performance.csv'

# Please verify these column names match your file
USER_ID_COL_ASSESS = 'course_learner_id'
GRADE_COL = 'grade'
MAX_GRADE_COL = 'max_grade'
# --- End of Configuration ---

try:
    # --- 1. Load and Preprocess Data ---
    print(f"--- Loading assessment data: {input_file_assess} ---")
    df_assess = pd.read_csv(input_file_assess)
    print(f"File loaded successfully! Found {len(df_assess)} assessment records.")

    required_cols_assess = [USER_ID_COL_ASSESS, GRADE_COL, MAX_GRADE_COL]
    if not all(col in df_assess.columns for col in required_cols_assess):
        raise KeyError(f"File is missing required columns. Needed {required_cols_assess}, but found {list(df_assess.columns)}")

    # Clean data: ensure grades are numeric and handle max_grade being zero
    df_assess[GRADE_COL] = pd.to_numeric(df_assess[GRADE_COL], errors='coerce')
    df_assess[MAX_GRADE_COL] = pd.to_numeric(df_assess[MAX_GRADE_COL], errors='coerce').replace(0, np.nan)
    df_assess.dropna(subset=[USER_ID_COL_ASSESS, GRADE_COL, MAX_GRADE_COL], inplace=True)
    print("Data cleaning complete.")

    # --- 2. Calculate Performance and Participation Features ---
    print("Grouping by student ID and calculating mean performance and participation count...")
    df_assess['score_percentage'] = df_assess[GRADE_COL] / df_assess[MAX_GRADE_COL]

    # Core step: Use .agg() to calculate two metrics at once
    feature_df_assess = df_assess.groupby(USER_ID_COL_ASSESS).agg(
        Overall_Performance=('score_percentage', 'mean'),
        Assessment_Participation_Count=(GRADE_COL, 'count')
    )
    feature_df_assess['Overall_Performance'] = (feature_df_assess['Overall_Performance'] * 100).round(2)
    print("Feature calculation complete!")

    # --- 3. Format and Save Results ---
    feature_df_assess = feature_df_assess.reset_index()
    print("\n--- Student Performance & Participation Features (First 10 rows) ---")
    print(feature_df_assess.head(10))
    feature_df_assess.to_csv(output_file_assess, index=False)
    print(f"\nSuccess! Results saved to '{output_file_assess}'.")

except FileNotFoundError:
    print(f"ERROR: File not found '{input_file_assess}'.")
except KeyError as e:
    print(f"ERROR: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

# ==============================================================================
# SCRIPT 2: FEATURE ENGINEERING - VIDEO ENGAGEMENT
# ==============================================================================
print("\n\n--- Running Script 2: Feature Engineering - Video Engagement ---")

# --- 0. Configuration ---
input_file_video = 'video_interactions_course-v1_KULeuvenX+EUROHISx+1T2023.csv'
output_file_video = 'feature_video_engagement.csv'
USER_ID_COL_VIDEO = 'course_learner_id'
DURATION_COL = 'duration'
PAUSE_COL = 'times_pause'
# --- End of Configuration ---

try:
    # --- 1. Load and Clean Data ---
    print(f"--- Loading data file: {input_file_video} ---")
    df_video = pd.read_csv(input_file_video)
    print(f"File loaded successfully! Found {len(df_video)} interaction records.")
    df_video[DURATION_COL] = pd.to_numeric(df_video[DURATION_COL], errors='coerce')
    df_video[PAUSE_COL] = pd.to_numeric(df_video[PAUSE_COL], errors='coerce')
    df_video.dropna(subset=[USER_ID_COL_VIDEO, DURATION_COL, PAUSE_COL], inplace=True)
    print("Data cleaning complete.")

    # --- 2. Calculate Features ---
    print(f"Grouping by '{USER_ID_COL_VIDEO}' and calculating total duration and pauses...")
    video_features_df = df_video.groupby(USER_ID_COL_VIDEO).agg(
        Total_Video_Duration_Seconds=(DURATION_COL, 'sum'),
        Total_Pause_Count=(PAUSE_COL, 'sum')
    )
    video_features_df['Total_Pause_Count'] = video_features_df['Total_Pause_Count'].astype(int)
    print("Calculation complete!")

    # --- 3. Format and Save Results ---
    video_features_df = video_features_df.reset_index()
    print("\n--- Video Interaction Features per User (First 10 rows) ---")
    print(video_features_df.head(10))
    video_features_df.to_csv(output_file_video, index=False)
    print(f"\nSuccess! Results saved to '{output_file_video}'.")

except FileNotFoundError:
    print(f"ERROR: File not found '{input_file_video}'.")
except KeyError as e:
    print(f"ERROR: Column not found {e}.")

# ==============================================================================
# SCRIPT 3: FEATURE ENGINEERING - FORUM ENGAGEMENT
# ==============================================================================
print("\n\n--- Running Script 3: Feature Engineering - Forum Engagement ---")

# --- 0. Configuration ---
input_file_forum = 'forum_interaction_course-v1_KULeuvenX+EUROHISx+1T2023.csv'
output_file_forum = 'feature_forum_engagement.csv'
USER_ID_COL_FORUM = 'course_learner_id'
BODY_COL = 'post_content'
# --- End of Configuration ---

try:
    # --- 1. Load and Preprocess Data ---
    print(f"--- Loading forum data: {input_file_forum} ---")
    df_forum = pd.read_csv(input_file_forum)
    print(f"File loaded successfully! Found {len(df_forum)} forum records.")
    df_forum[BODY_COL].fillna('', inplace=True)
    df_forum['post_content_length'] = df_forum[BODY_COL].str.len()
    df_forum.dropna(subset=[USER_ID_COL_FORUM], inplace=True)
    print("Data preprocessing complete.")

    # --- 2. Calculate Forum Engagement Features ---
    print("Grouping by student ID and calculating total post count and average post length...")
    feature_df_forum = df_forum.groupby(USER_ID_COL_FORUM).agg(
        Total_Post_Count=(BODY_COL, 'count'),
        Average_Post_Length=('post_content_length', 'mean')
    )
    feature_df_forum['Average_Post_Length'] = feature_df_forum['Average_Post_Length'].round(1)
    print("Feature calculation complete!")

    # --- 3. Format and Save Results ---
    feature_df_forum = feature_df_forum.reset_index()
    print("\n--- Forum Engagement Features per User (First 10 rows) ---")
    print(feature_df_forum.head(10))
    feature_df_forum.to_csv(output_file_forum, index=False)
    print(f"\nSuccess! Results saved to '{output_file_forum}'.")

except FileNotFoundError:
    print(f"ERROR: File not found '{input_file_forum}'.")
except KeyError as e:
    print(f"ERROR: Column not found {e}.")

# ==============================================================================
# SCRIPT 4: FEATURE ENGINEERING - TEMPORAL PATTERNS (ACTIVE WEEKS)
# ==============================================================================
print("\n\n--- Running Script 4: Feature Engineering - Active Weeks ---")

# --- 0. Configuration ---
input_file_sessions = 'sessions_course-v1_KULeuvenX+EUROHISx+1T2023.csv'
output_file_weeks = 'feature_active_weeks_per_user.csv'
USER_ID_COL_SESSIONS = 'course_learner_id'
TIMESTAMP_COL = 'start_time'
# --- End of Configuration ---

try:
    # --- 1. Load and Preprocess Data ---
    print(f"--- Loading session data: {input_file_sessions} ---")
    df_sessions = pd.read_csv(input_file_sessions)
    print(f"File loaded successfully! Found {len(df_sessions)} session records.")
    print("Cleaning timestamp format...")
    df_sessions[TIMESTAMP_COL] = df_sessions[TIMESTAMP_COL].str.replace(r' GMT\+\d{4} \(.*\)', '', regex=True)
    df_sessions[TIMESTAMP_COL] = pd.to_datetime(df_sessions[TIMESTAMP_COL], format='%a %b %d %Y %H:%M:%S', errors='coerce')
    df_sessions.dropna(subset=[USER_ID_COL_SESSIONS, TIMESTAMP_COL], inplace=True)
    print("Data preprocessing complete.")

    # --- 2. Calculate Active Weeks per User ---
    print(f"Grouping by '{USER_ID_COL_SESSIONS}' and calculating unique weeks...")
    df_sessions['activity_week'] = df_sessions[TIMESTAMP_COL].dt.to_period('W')
    active_weeks_per_user = df_sessions.groupby(USER_ID_COL_SESSIONS)['activity_week'].nunique()

    # --- 3. Format and Save Results ---
    active_weeks_df = active_weeks_per_user.reset_index()
    active_weeks_df.columns = [USER_ID_COL_SESSIONS, 'Number_of_Active_Weeks']
    print("Calculation complete!")
    print("\n--- Active Weeks per User (First 10 rows) ---")
    print(active_weeks_df.head(10))
    active_weeks_df.to_csv(output_file_weeks, index=False)
    print(f"\nSuccess! Results saved to '{output_file_weeks}'.")

except FileNotFoundError:
    print(f"ERROR: File not found '{input_file_sessions}'.")
except KeyError as e:
    print(f"ERROR: Column not found {e}.")

# ==============================================================================
# SCRIPT 5: MERGE ALL FEATURES INTO FINAL MATRIX
# ==============================================================================
print("\n\n--- Running Script 5: Merging All Features ---")

# --- 0. Configuration ---
USER_ID_COL_MERGE = 'course_learner_id'
output_file_matrix = 'final_feature_matrix_for_clustering.csv'
# --- End of Configuration ---

try:
    # --- 1. Load All Individual Feature Files ---
    print("--- Loading all feature files ---")
    df_assess = pd.read_csv(output_file_assess)
    print(f" -> Loaded assessment features for {len(df_assess)} students.")
    df_forum = pd.read_csv(output_file_forum)
    print(f" -> Loaded forum features for {len(df_forum)} students.")
    df_video = pd.read_csv(output_file_video)
    print(f" -> Loaded video features for {len(df_video)} students.")
    df_weeks = pd.read_csv(output_file_weeks)
    print(f" -> Loaded active weeks feature for {len(df_weeks)} students.")

    # --- 2. Chained Merge ---
    print("\n--- Merging all features into a single matrix ---")
    # Use 'outer' merge to keep all students who appear in at least one file
    merged_df = pd.merge(df_weeks, df_video, on=USER_ID_COL_MERGE, how='outer')
    merged_df = pd.merge(merged_df, df_forum, on=USER_ID_COL_MERGE, how='outer')
    final_feature_matrix = pd.merge(merged_df, df_assess, on=USER_ID_COL_MERGE, how='outer')
    print("Initial merge complete. Now handling missing values...")

    # --- 3. Fill Missing Values ---
    # Critical step: NaNs resulting from an outer merge mean "no participation" for that feature.
    # We replace these NaNs with 0.
    final_feature_matrix.fillna(0, inplace=True)
    print("Missing values filled with 0.")

    # --- 4. Display and Save Final Result ---
    print("\n--- Final Feature Matrix Preview (First 10 rows) ---")
    print(final_feature_matrix.head(10))
    print(f"\nFinal matrix contains {len(final_feature_matrix)} students and {len(final_feature_matrix.columns)} features.")
    print("Columns:", list(final_feature_matrix.columns))
    final_feature_matrix.to_csv(output_file_matrix, index=False)
    print(f"\nSuccess! Final feature matrix saved to '{output_file_matrix}'.")

except FileNotFoundError as e:
    print(f"\nERROR: File not found {e.filename}. Ensure all feature files were generated correctly.")
except Exception as e:
    print(f"An error occurred during processing: {e}")

# ==============================================================================
# SCRIPT 6: K-MEANS CLUSTERING
# ==============================================================================
print("\n\n--- Running Script 6: K-Means Clustering ---")

# --- 0. Configuration ---
input_file_cluster = 'final_feature_matrix_for_clustering.csv'
USER_ID_COL_CLUSTER = 'course_learner_id'
output_labels_file = 'student_cluster_labels.csv'
# --- End of Configuration ---

try:
    # --- 1. Load and Prepare Data ---
    print("--- Step 1: Loading feature matrix ---")
    df_cluster = pd.read_csv(input_file_cluster)
    user_ids = df_cluster[USER_ID_COL_CLUSTER]
    features = df_cluster.drop(columns=[USER_ID_COL_CLUSTER])
    print(f"Data loaded successfully! Clustering {len(features.columns)} features for {len(features)} students.")

    # --- 2. Feature Standardization ---
    print("\n--- Step 2: Standardizing features ---")
    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(features)
    print("Standardization complete.")

    # --- 3. Find Optimal K using Silhouette Score (Optional) ---
    print("\n--- Step 3: Finding optimal K using Silhouette Score ---")
    k_range = range(2, 11)
    silhouette_scores = []
    for k in k_range:
        kmeans = KMeans(n_clusters=k, init='k-means++', random_state=42, n_init=10)
        kmeans.fit(features_scaled)
        score = silhouette_score(features_scaled, kmeans.labels_)
        silhouette_scores.append(score)
    best_k = k_range[np.argmax(silhouette_scores)]
    print(f"Diagnosis: Highest silhouette score found at K = {best_k}.")

    # Plot silhouette scores
    plt.figure(figsize=(10, 6))
    plt.plot(k_range, silhouette_scores, marker='o')
    plt.title('Silhouette Score for Optimal K')
    plt.xlabel('Number of Clusters (K)')
    plt.ylabel('Silhouette Score')
    plt.grid(True)
    plt.show()

    # --- 4. Perform Final Clustering ---
    optimal_k = 2  # Set K=2 based on the analysis
    print(f"\n--- Step 4: Performing final clustering with K = {optimal_k} ---")
    kmeans_final = KMeans(n_clusters=optimal_k, init='k-means++', random_state=42, n_init=10)
    final_labels = kmeans_final.fit_predict(features_scaled)
    df_cluster['Cluster'] = final_labels
    print("Clustering complete.")

    # --- 5. Interpret Cluster Centroids ---
    print("\n--- Step 5: Interpreting cluster centroids ---")
    centroids_original_scale = scaler.inverse_transform(kmeans_final.cluster_centers_)
    centroid_df = pd.DataFrame(centroids_original_scale, columns=features.columns)
    centroid_df.index.name = 'Cluster'
    print(centroid_df.round(1))

    # --- 6. Visualize Clusters with PCA (Optional) ---
    print("\n--- Step 6: Visualizing clusters with PCA ---")
    pca = PCA(n_components=2)
    features_pca = pca.fit_transform(features_scaled)
    df_pca = pd.DataFrame(data=features_pca, columns=['PC1', 'PC2'])
    df_pca['Cluster'] = final_labels
    plt.figure(figsize=(12, 8))
    sns.scatterplot(x='PC1', y='PC2', hue='Cluster', palette=sns.color_palette("hls", optimal_k), data=df_pca, legend="full")
    plt.title('K-Means Clustering of Learners (PCA Visualization)')
    plt.show()

    # --- 7. Generate and Save Final Student Labels ---
    print("\n--- Step 7: Generating final student label file ---")
    final_student_labels_df = df_cluster[[USER_ID_COL_CLUSTER, 'Cluster']]
    # Map cluster numbers to meaningful names based on centroid interpretation
    cluster_names = {0: 'More Active Learners', 1: 'Less Active Learners'} # Adjust based on your interpretation
    final_student_labels_df['Cluster_Name'] = final_student_labels_df['Cluster'].map(cluster_names)
    final_student_labels_df.to_csv(output_labels_file, index=False)
    print(f"\nSuccess! Student cluster labels saved to '{output_labels_file}'.")
    print("File preview:")
    print(final_student_labels_df.head())

except FileNotFoundError:
    print(f"ERROR: File not found '{input_file_cluster}'.")
except Exception as e:
    print(f"An error occurred during processing: {e}")

# ==============================================================================
# SCRIPT 7: FILTER FOR PASSING STUDENTS
# ==============================================================================
print("\n\n--- Running Script 7: Filtering for Passing Students ---")

# --- 0. Configuration ---
input_file_roster = 'course_learner_course-v1_KULeuvenX+EUROHISx+1T2023.csv'
output_file_passing = 'passing_students_only.csv'
CERTIFICATE_STATUS_COL = 'certificate_status'
# --- End of Configuration ---

try:
    # --- 1. Load Data ---
    print(f"--- Loading data file: {input_file_roster} ---")
    df_roster = pd.read_csv(input_file_roster)
    print(f"File loaded successfully! Found {len(df_roster)} records.")

    # --- 2. Filter Data ---
    if CERTIFICATE_STATUS_COL not in df_roster.columns:
        raise KeyError(f"Column '{CERTIFICATE_STATUS_COL}' not found. Available columns: {list(df_roster.columns)}")
    print(f"\n--- Filtering for records where '{CERTIFICATE_STATUS_COL}' is 'downloadable' ---")
    df_passing = df_roster[df_roster[CERTIFICATE_STATUS_COL] == 'downloadable'].copy()
    print("Filtering complete.")
    print(f"Original records: {len(df_roster)}")
    print(f"Records with 'downloadable' certificate: {len(df_passing)}")

    # --- 3. Display and Save Results ---
    print("\n--- Preview of passing students (First 5 rows) ---")
    print(df_passing.head())
    df_passing.to_csv(output_file_passing, index=False)
    print(f"\nSuccess! Filtered data saved to '{output_file_passing}'.")

except FileNotFoundError:
    print(f"ERROR: File not found '{input_file_roster}'.")
except KeyError as e:
    print(f"ERROR: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

# ==============================================================================
# SCRIPT 8: VALIDATE CLUSTERS WITH PASSING STATUS
# ==============================================================================
print("\n\n--- Running Script 8: Validating Clusters ---")

# --- 0. Configuration ---
clusters_file = 'student_cluster_labels.csv'
passing_file = 'passing_students_only.csv'
USER_ID_COL_VALIDATE = 'course_learner_id'
# --- End of Configuration ---

try:
    # --- 1. Load Data ---
    print("--- Step 1: Loading data files ---")
    df_clusters = pd.read_csv(clusters_file)
    print(f" -> Loaded cluster labels for {len(df_clusters)} students.")
    df_passing = pd.read_csv(passing_file)
    print(f" -> Loaded passing roster for {len(df_passing)} students.")

    # --- 2. Prepare and Merge Data ---
    print("\n--- Step 2: Merging data and marking passing/not passing status ---")
    df_passing['Status'] = 'Passing'
    # Use a left merge to keep all students from the clustering analysis
    merged_df_validate = pd.merge(df_clusters, df_passing[[USER_ID_COL_VALIDATE, 'Status']], on=USER_ID_COL_VALIDATE, how='left')
    # Fill NaNs with 'Not Passing' for students not in the passing list
    merged_df_validate['Status'].fillna('Not Passing', inplace=True)
    print("Data merge and status marking complete.")

    # --- 3. Create Contingency Table ---
    print("\n--- Step 3: Generating contingency table ---")
    contingency_table = pd.crosstab(
        index=merged_df_validate['Cluster_Name'],
        columns=merged_df_validate['Status']
    )
    print("Contingency table generated successfully!")
    print(contingency_table)

    # --- 4. Calculate Pass Rates ---
    print("\n--- Step 4: Calculating pass rates for each cluster ---")
    # Calculate pass rate for each cluster name found in the table
    for cluster_name in contingency_table.index:
        passing_count = contingency_table.loc[cluster_name, 'Passing']
        total_count = contingency_table.loc[cluster_name].sum()
        pass_rate = (passing_count / total_count) * 100
        print(f"Pass rate for '{cluster_name}': {pass_rate:.1f}%")

except FileNotFoundError as e:
    print(f"\nERROR: File not found {e.filename}. Please ensure both CSV files are available.")
except KeyError as e:
    print(f"\nERROR: Column not found {e}. This often means the user ID column names do not match.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

# ==============================================================================
# SCRIPT 9: K-MEANS ROBUSTNESS CHECK (50 RUNS)
# ==============================================================================
print("\n\n--- Running Script 9: K-Means Robustness Check ---")

# --- 0. Configuration ---
# NOTE: This script needs to be run for each course by changing the input file.
input_file_robustness = 'final_feature_matrix_for_clustering.csv'
USER_ID_COL_ROBUSTNESS = 'course_learner_id'
summary_output_file = 'cluster_robustness_summary.csv'
N_RUNS = 50
OPTIMAL_K_ROBUSTNESS = 2
# --- End of Configuration ---

try:
    # --- 1. Load and Prepare Data ---
    print("--- Step 1: Loading feature matrix for robustness check ---")
    df_robustness_data = pd.read_csv(input_file_robustness)
    features_robustness = df_robustness_data.drop(columns=[USER_ID_COL_ROBUSTNESS])
    print(f"Data loaded successfully!")

    # --- 2. Feature Standardization ---
    print("\n--- Step 2: Standardizing features ---")
    scaler_robustness = StandardScaler()
    features_scaled_robustness = scaler_robustness.fit_transform(features_robustness)
    print("Standardization complete.")

    # --- 3. Run Clustering 50 Times ---
    cluster_size_results = []
    print(f"Running K-Means {N_RUNS} times with K={OPTIMAL_K_ROBUSTNESS} to check for stability...")

    for i in range(N_RUNS):
        # Key: Use a different random_state for each iteration
        kmeans_robustness = KMeans(n_clusters=OPTIMAL_K_ROBUSTNESS, init='k-means++', random_state=i, n_init=10)
        labels = kmeans_robustness.fit_predict(features_scaled_robustness)

        # Count the size of each cluster
        cluster_sizes = pd.Series(labels).value_counts().sort_index()

        # Store the results in a list
        result_row = {'run': i + 1}
        for cluster_id in range(OPTIMAL_K_ROBUSTNESS):
            result_row[f'cluster_{cluster_id}_size'] = cluster_sizes.get(cluster_id, 0)
        cluster_size_results.append(result_row)

    # --- 4. Analyze and Save Results ---
    df_robustness_results = pd.DataFrame(cluster_size_results)
    print("\n--- Robustness Check Results ---")
    print(f"Completed {N_RUNS} clustering runs. Descriptive statistics for cluster sizes are as follows:")

    # Calculate and print descriptive statistics for the appendix
    cluster_size_summary = df_robustness_results.drop(columns=['run']).describe()
    print(cluster_size_summary.round(2))

    # --- End of Robustness Check ---

except FileNotFoundError:
    print(f"ERROR: File not found '{input_file_robustness}'.")
except Exception as e:
    print(f"An error occurred during processing: {e}")